In [8]:
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from functools import reduce
from selenium.common.exceptions import TimeoutException

import mysql.connector as mysql
from mysql.connector import Error

import pandas as pd
import time
import datetime
import re

In [1]:
data = pd.read_csv("D:/Images2/Population_FS.csv")
print(data)

conn = mysql.connect(host='localhost', database='****', user='newuser', password='l******d', auth_plugin='mysql_native_password')
if conn.is_connected():
    cursor = conn.cursor()
    cursor.execute("select database();")
    record = cursor.fetchone()

'data = pd.read_csv("D:/Images2/Population_FS.csv")\nprint(data)\n\nconn = mysql.connect(host=\'localhost\', database=\'****\', user=\'newuser\', password=\'l******d\', auth_plugin=\'mysql_native_password\')\nif conn.is_connected():\n    cursor = conn.cursor()\n    cursor.execute("select database();")\n    record = cursor.fetchone()'

In [2]:
while True:    
    options = uc.ChromeOptions()
    options.add_argument("start-maxmized")
    driver = uc.Chrome(options=options)
    driver.maximize_window()

    url = "https://www.familysearch.org/search/record/results?q.anyDate.from=1000&q.anyDate.to=9999&f.collectionId=1661470"
    driver.get("https://www.familysearch.org/search/record/results?q.anyDate.from=1000&q.anyDate.to=9999&f.collectionId=1661470")

    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="userName"]'))).send_keys("****")
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="password"]'))).send_keys("****")
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="login"]'))).click()


    try:
        for index, i in data.iterrows():
            
            time.sleep(5)
            driver.get(i['url'])

            row = {}

            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="resultsContainer"]/div/div/div/div/div/div[1]/div[1]/div/div[2]/div/div/div[1]')))
            time.sleep(2)
            population = driver.find_element(By.XPATH, '//*[@id="resultsContainer"]/div/div/div/div/div/div[1]/div[1]/div/div[2]/div/div/div[1]').text

            row['URL'] = i['url']
            row['population'] = population
            
            if row['population'] != "":
                
                sql = 'INSERT INTO `population` VALUES(%s, %s)'
                cursor.execute(sql, tuple(row.values()))

                conn.commit()

                print(tuple(row.values()))

                data.drop(index, inplace=True)
            
            else:
                
                driver.refresh()
                time.sleep(8)
                
                driver.get(i['url'])

                row = {}

                WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="resultsContainer"]/div/div/div/div/div/div[1]/div[1]/div/div[2]/div/div/div[1]')))
                time.sleep(2)
                population = driver.find_element(By.XPATH, '//*[@id="resultsContainer"]/div/div/div/div/div/div[1]/div[1]/div/div[2]/div/div/div[1]').text

                row['URL'] = i['url']
                row['population'] = population
                
                sql = 'INSERT INTO `population` VALUES(%s, %s)'
                cursor.execute(sql, tuple(row.values()))

                conn.commit()

                print(tuple(row.values()))

                data.drop(index, inplace=True)
                
    except TimeoutException:
        driver.refresh()
        time.sleep(8)
        continue
    

'while True:    \n    options = uc.ChromeOptions()\n    options.add_argument("start-maxmized")\n    driver = uc.Chrome(options=options)\n    driver.maximize_window()\n\n    url = "https://www.familysearch.org/search/record/results?q.anyDate.from=1000&q.anyDate.to=9999&f.collectionId=1661470"\n    driver.get("https://www.familysearch.org/search/record/results?q.anyDate.from=1000&q.anyDate.to=9999&f.collectionId=1661470")\n\n    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, \'//*[@id="userName"]\'))).send_keys("wendycyw")\n    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, \'//*[@id="password"]\'))).send_keys("lifewood123")\n    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, \'//*[@id="login"]\'))).click()\n\n\n    try:\n        for index, i in data.iterrows():\n            \n            time.sleep(5)\n            driver.get(i[\'url\'])\n\n            row = {}\n\n            WebDriverWait(driver, 30).until(EC.presence_